# Cold Email Generator

In [1]:
from langchain_groq import ChatGroq

In [2]:
llm = ChatGroq(
    temperature=0,
    groq_api_key='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',
    model_name="llama-3.1-70b-versatile",
)

response = llm.invoke(" The first footballer to win 3 Champions League titles with 3 different teams is ... ")
print(response.content)

The answer is Clarence Seedorf.


In [3]:
from langchain_community.document_loaders import WebBaseLoader

loader = WebBaseLoader("https://jobs.nike.com/fr/job/R-47136?from=job%20search%20funnel")

page_data= loader.load().pop().page_content
print(page_data)

USER_AGENT environment variable not set, consider setting it to identify your requests.


Postuler pour Software Engineer

Rechercher des postesPasser la navigationRechercher des postesNIKE, INC. JOBSEmplois temporairesJoin The Talent CommunityNike au quotidienPrésentationAvantages sociauxMarquesPrésentationJordanConverseÉquipesPrésentationSoutien administratifInnovation de pointeAir Manufacturing et InnovationAéronautiqueCommunicationsService clientConceptionNumériqueInstallationsFinance et comptabilitéAffaires gouvernementales et publiquesRessources humainesDonnées et analyseService juridiqueFabrication et ingénierieMarketingMerchandisingPlanificationConfidentialitéAchatsCréation, développement et gestion de produitsService de gestion des magasinsMagasinsVentesImpact social et communautaireMarketing sportsPlanification stratégiqueChaîne d'approvisionnement, distribution et logistiqueDurabilitéTechnologieSitesPrésentationNike WHQSiège de Nike à New YorkEHQ : Hilversum, Pays-BasELC: Laakdal, BelgiumGreater China HQDiversité, égalité et inclusivitéPrésentationMilitaryHandica

In [4]:
from langchain_core.prompts import PromptTemplate


In [5]:
prompt_extract = PromptTemplate.from_template(
    """
    ### SCRAPED TEXT FROM WEBSITE:
    {page_data}
    ### INSTRUCTION:
    The scraped text is from the career's page of a website.
    Your job is to extract the job postings and return them in JSON format containing the 
    following keys: 'role', 'experience','skills' and 'description'.
    Only return the valid JSON.
    ### VALID JSON (NO PREAMBLE):
    """
)
chain_extract = prompt_extract | llm
res = chain_extract.invoke(input={'page_data':page_data})
print(res.content)

[
  {
    "role": "Software Engineer",
    "experience": "5 years of progressive post-baccalaureate experience in the job offered or in a Engineering-related occupation",
    "skills": [
      "Java",
      "Javascript",
      "NodeJS",
      "Groovy",
      "AWS Cloud platform",
      "REST API",
      "Automation scripting frameworks and technologies such as Testcafe, Selenium or Postman/Rest-assured",
      "Performance tools such as Jmeter or Gatling",
      "Build Management tool such as Gradle",
      "Version Control tools such as Github",
      "SAP modules order management",
      "Dev-ops tools such as Splunk, signalFX, AWS Cloudwatch, and Jenkins",
      "Database tools such as Mysql, PostgreSQL, and Teradata",
      "Cloud testing platform such as browserstack"
    ],
    "description": "Develop, code, configure, and test programs and systems and solutions problem in order to meet designed digital product specification and direction. Manage the team on gathering detail desi

In [6]:
from langchain_core.output_parsers import JsonOutputParser

json_parser = JsonOutputParser()
json_res = json_parser.parse(res.content)
json_res

[{'role': 'Software Engineer',
  'experience': '5 years of progressive post-baccalaureate experience in the job offered or in a Engineering-related occupation',
  'skills': ['Java',
   'Javascript',
   'NodeJS',
   'Groovy',
   'AWS Cloud platform',
   'REST API',
   'Automation scripting frameworks and technologies such as Testcafe, Selenium or Postman/Rest-assured',
   'Performance tools such as Jmeter or Gatling',
   'Build Management tool such as Gradle',
   'Version Control tools such as Github',
   'SAP modules order management',
   'Dev-ops tools such as Splunk, signalFX, AWS Cloudwatch, and Jenkins',
   'Database tools such as Mysql, PostgreSQL, and Teradata',
   'Cloud testing platform such as browserstack'],
  'description': 'Develop, code, configure, and test programs and systems and solutions problem in order to meet designed digital product specification and direction. Manage the team on gathering detail design and solving complex problems. Ensure tech design driven by the

In [7]:
import pandas as pd

df = pd.read_csv("my_portfolio.csv")
df

,Techstack,Links
0,"React, Node.js, MongoDB",https://example.com/react-portfolio
1,"Angular,.NET, SQL Server",https://example.com/angular-portfolio
2,"Vue.js, Ruby on Rails, PostgreSQL",https://example.com/vue-portfolio
3,"Python, Django, MySQL",https://example.com/python-portfolio
4,"Java, Spring Boot, Oracle",https://example.com/java-portfolio
5,"Flutter, Firebase, GraphQL",https://example.com/flutter-portfolio
6,"WordPress, PHP, MySQL",https://example.com/wordpress-portfolio
7,"Magento, PHP, MySQL",https://example.com/magento-portfolio
8,"React Native, Node.js, MongoDB",https://example.com/react-native-portfolio
9,"iOS, Swift, Core Data",https://example.com/ios-portfolio


In [8]:
import uuid
import chromadb

client = chromadb.PersistentClient('vectorstore')
collection = client.get_or_create_collection(name="portfolio")

if not collection.count():
    for _, row in df.iterrows():
        collection.add(documents=row["Techstack"],
                       metadatas={"links": row["Links"]},
                       ids=[str(uuid.uuid4())])

In [12]:
links = collection.query(query_texts=job['skills'], n_results=2).get('metadatas', [])
links

[[{'links': 'https://example.com/java-portfolio'},
  {'links': 'https://example.com/android-portfolio'}],
 [{'links': 'https://example.com/full-stack-js-portfolio'},
  {'links': 'https://example.com/typescript-frontend-portfolio'}],
 [{'links': 'https://example.com/react-portfolio'},
  {'links': 'https://example.com/react-native-portfolio'}],
 [{'links': 'https://example.com/kotlin-backend-portfolio'},
  {'links': 'https://example.com/java-portfolio'}],
 [{'links': 'https://example.com/xamarin-portfolio'},
  {'links': 'https://example.com/devops-portfolio'}],
 [{'links': 'https://example.com/vue-portfolio'},
  {'links': 'https://example.com/wordpress-portfolio'}],
 [{'links': 'https://example.com/full-stack-js-portfolio'},
  {'links': 'https://example.com/devops-portfolio'}],
 [{'links': 'https://example.com/ml-python-portfolio'},
  {'links': 'https://example.com/vue-portfolio'}],
 [{'links': 'https://example.com/devops-portfolio'},
  {'links': 'https://example.com/kotlin-backend-portf

In [9]:
job = json_res
job['skills']

TypeError: list indices must be integers or slices, not str

In [13]:
prompt_email = PromptTemplate.from_template(
        """
        ### JOB DESCRIPTION:
        {job_description}
        
        ### INSTRUCTION:
        You are Mohan, a business development executive at AtliQ. AtliQ is an AI & Software Consulting company dedicated to facilitating
        the seamless integration of business processes through automated tools. 
        Over our experience, we have empowered numerous enterprises with tailored solutions, fostering scalability, 
        process optimization, cost reduction, and heightened overall efficiency. 
        Your job is to write a cold email to the client regarding the job mentioned above describing the capability of AtliQ 
        in fulfilling their needs.
        Also add the most relevant ones from the following links to showcase Atliq's portfolio: {link_list}
        Remember you are Mohan, BDE at AtliQ. 
        Do not provide a preamble.
        ### EMAIL (NO PREAMBLE):
        
        """
        )

chain_email = prompt_email | llm
res = chain_email.invoke({"job_description": str(job), "link_list": links})
print(res.content)

Subject: Expert Software Engineering Services for NIKE's Digital Products

Dear Hiring Manager,

I came across the job description for a Software Engineer at NIKE, and I'm excited to introduce AtliQ, a leading AI & Software Consulting company. Our team of experts has extensive experience in developing, coding, configuring, and testing programs and systems that meet the highest standards of digital product specifications.

At AtliQ, we have a proven track record of delivering scalable, efficient, and cost-effective solutions that cater to the needs of our clients. Our expertise in Java, Javascript, NodeJS, Groovy, and AWS Cloud platform aligns perfectly with the requirements mentioned in the job description. We also have experience with Automation scripting frameworks and technologies such as Testcafe, Selenium, and Postman/Rest-assured.

Our team has worked on various projects that involve designing and building components, frameworks, and libraries at scale to support complex systems.